In [78]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent, create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import OpenAI
from langchain import hub
from langchain_openai import ChatOpenAI

In [89]:
%pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [93]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
display(os.environ.get("NEO4J_USERNAME"))

'neo4j'

In [94]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_NAME"] = "agentmem"
os.environ["WANDB_NOTES"] = "memoryagent"
os.environ["WANDB_PROJECT"] = "langchain-tracing"
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import wandb_tracing_enabled
from langchain_openai import OpenAI

In [95]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = GoogleSearchAPIWrapper()
# tools = [
#     Tool(
#         name="Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events",
#     )
# ]
tools = [TavilySearchResults(max_results=1)]

In [1]:
from IPython.display import Markdown
from langchain.agents.load_tools import get_all_tool_names
from langchain import hub
prompt = hub.pull("hwchase17/react-chat")

display(Markdown(prompt.template))
display(prompt.input_variables)


Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}

['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools']

In [104]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

# prompt = ZeroShotAgent.create_prompt(
#     tools,
#     prefix=prefix,
#     suffix=suffix,
#     input_variables=["input", "chat_history", "agent_scratchpad"],
# )
# display(Markdown(prompt.template))
# display(prompt.input_variables)

prompttemplate = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

PROMPT = PromptTemplate.from_template(prompttemplate)
display(Markdown(PROMPT.template))
display(PROMPT.input_variables)

Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}

['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools']

In [105]:
message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history
)

In [106]:
llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0.4)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [107]:
agent_chain.invoke({"input":"How many people live in canada?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: How many people live in Canada?[{'url': 'https://www.canadapopulation.net/', 'content': "Population of Canada 2024 Canada Population 2024  more about Quebec population 2024.  Toronto Population 2024.  more about Nova Scotia population 2024.We believe using the recent years' figures (see the table below) will make the estimation more accurate. Population Growth of Canada The following table shows Canada is enjoying healthy and consistent population growth over last 12 years range from 0.92% to 1.00% adding 329,278 to 346,537 people to the overall population each year."}]Do I need to use a tool? No
Final Answer: The search results did not provide the current population of Canada but indicated that Canada has been experiencing a healthy and consistent population growth ranging from 0.92% to 1.00%, adding between 329,278 to 346,537 people to the overall population 

{'input': 'How many people live in canada?',
 'chat_history': '',
 'output': 'The search results did not provide the current population of Canada but indicated that Canada has been experiencing a healthy and consistent population growth ranging from 0.92% to 1.00%, adding between 329,278 to 346,537 people to the overall population each year. For the most current population figure, it would be best to refer to the latest statistics from a reliable source such as Statistics Canada or a similar demographic authority.'}

In [109]:
# Now, we unset the environment variable and use a context manager.
if "LANGCHAIN_WANDB_TRACING" in os.environ:
    del os.environ["LANGCHAIN_WANDB_TRACING"]

# enable tracing using a context manager
# with wandb_tracing_enabled():
#     agent.run("What is 5 raised to .123243 power?")  # this should be traced

# agent.run("What is 2 raised to .123243 power?")  # this should not be traced

In [110]:
with wandb_tracing_enabled():
    agent_chain.invoke({"input":"what is their national anthem called?"})



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: The national anthem of Canada is called "O Canada."
```

> Finished chain.
